# Origen de los datos

Los datos para el presente trabajo serán los datos de los vuelos producidos a nivel mundial entre Enero de 2019 y Abril de 2021, obtenidos a partir de la página web **Open Sky Network** (https://opensky-network.org/) y disponibles de manera *open source* en el siguiente enlace: https://zenodo.org/record/4670228#.YJLBKbUzZPZ

# Análisis y limpieza de los datos

La preparación de los datos empieza por renombrar los archivos de manera que sea posible la enumeración de cada *dataset* de la pàgina web **Open Sky Network**. En este caso enumeramos del **1** al **28**.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv('input_data_raw/1.csv.gz',sep=',',encoding='utf-8')

C:\Users\AL191\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df.head()

,callsign,number,icao24,registration,typecode,origin,destination,firstseen,lastseen,day,latitude_1,longitude_1,altitude_1,latitude_2,longitude_2,altitude_2
0,HVN19,NaN,888152,NaN,NaN,YMML,LFPG,2018-12-31 00:43:16+00:00,2019-01-01 04:56:29+00:00,2019-01-01 00:00:00+00:00,-37.659485,144.804421,304.8,48.995316,2.610802,-53.34
1,CCA839,NaN,780ad1,NaN,NaN,YMML,LEBL,2018-12-31 00:53:08+00:00,2019-01-01 06:01:42+00:00,2019-01-01 00:00:00+00:00,-37.692123,144.841997,304.8,41.036124,2.063557,1112.52
2,CES219,NaN,780b7e,B-5936,A332,YSSY,EDDF,2018-12-31 01:05:29+00:00,2019-01-01 04:09:29+00:00,2019-01-01 00:00:00+00:00,-33.930908,151.171987,0.0,50.045563,8.588923,-15.24
3,AEA040,NaN,34444e,EC-LVL,A332,LEMD,LEMD,2018-12-31 01:07:21+00:00,2019-01-01 03:32:59+00:00,2019-01-01 00:00:00+00:00,40.534756,-3.575426,609.6,40.475728,-3.538347,411.48
4,CXA825,NaN,780d75,B-2760,B788,YSSY,LFPG,2018-12-31 01:18:29+00:00,2019-01-01 04:32:28+00:00,2019-01-01 00:00:00+00:00,-33.954254,151.178041,0.0,48.996091,2.625805,22.86


### **Descripción del Data Set**

En la web mencionada anteriormente, se presentan las descripciones de las variables observadas:


One file per month is provided as a csv file with the following features:

- **callsign**: the identifier of the flight displayed on ATC screens (usually the first three letters are reserved for an airline: AFR for Air France, DLH for Lufthansa, etc.)
- **number**: the commercial number of the flight, when available (the matching with the callsign comes from public open API)
- **icao24**: the transponder unique identification number;
- **registration**: the aircraft tail number (when available);
- **typecode**: the aircraft model type (when available);
- **origin**: a four letter code for the origin airport of the flight (when available);
- **destination**: a four letter code for the destination airport of the flight (when available);
- **firstseen**: the UTC timestamp of the first message received by the OpenSky Network;
- **lastseen**: the UTC timestamp of the last message received by the OpenSky Network;
- **day**: the UTC day of the last message received by the OpenSky Network;
- **latitude_1, longitude_1, altitude_1**: the first detected position of the aircraft;
- **latitude_2, longitude_2, altitude_2**: the last detected position of the aircraft.


### **Limpieza de datos**

Se observa que cada uno de los ficheros ocupa casi 0.5 GB, pero no toda la información es relevante para el estudio que se pretende, en este caso, se va a prescindir de las siguientes columnas, ya sea por información redundante o irrelevante: ***number, icao24, registration, typecode, firstseen y lastseen***. Permanecen entonces los campos ***callsign*, *origin*, *destination*, *day*, *altitude_1*, *altitude_2*, *longitude_1*** y ***longitude_2***, con el objetivo de facilitar el procesamiento de los mismos.

In [6]:
df.drop(labels=['number','icao24','registration','typecode','firstseen','lastseen','altitude_1','altitude_2'], 
        axis=1,
        inplace=True)

In [6]:
df.head()

,callsign,origin,destination,day,latitude_1,longitude_1,latitude_2,longitude_2
0,HVN19,YMML,LFPG,2019-01-01 00:00:00+00:00,-37.659485,144.804421,48.995316,2.610802
1,CCA839,YMML,LEBL,2019-01-01 00:00:00+00:00,-37.692123,144.841997,41.036124,2.063557
2,CES219,YSSY,EDDF,2019-01-01 00:00:00+00:00,-33.930908,151.171987,50.045563,8.588923
3,AEA040,LEMD,LEMD,2019-01-01 00:00:00+00:00,40.534756,-3.575426,40.475728,-3.538347
4,CXA825,YSSY,LFPG,2019-01-01 00:00:00+00:00,-33.954254,151.178041,48.996091,2.625805


### Análisis y preprocesamiento de los datos

Para comprender los datos tomados como input se va a realizar un estudio a distintos niveles de profundidad. En este notebook, a modo de ejemplo, se realizará el estudio de un mes de vuelos dada la cantidad total de datos y considerando que la muestra es representativa. En el siguiente apartado del trabajo se encontrarán resumidos y automatizados la metodología contenida en este notebook para el resto de meses disponibles obteniendo finalmente un data set limpio y completo para los 28 meses de estudio.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2145469 entries, 0 to 2145468
Data columns (total 8 columns):
 #   Column       Dtype  
---  ------       -----  
 0   callsign     object 
 1   origin       object 
 2   destination  object 
 3   day          object 
 4   latitude_1   float64
 5   longitude_1  float64
 6   latitude_2   float64
 7   longitude_2  float64
dtypes: float64(4), object(4)
memory usage: 130.9+ MB


Solamente son de tipo float las variables de latitud y longitud y el resto son de tipo string. Se destaca también que el campo  ***day*** también es tipo string y no tipo ***datetime*** como se esperaría.

In [8]:
print('Porcentaje de nulos por columna:')
for e in df.columns: 
    print(e,np.round(df[e].isnull().sum()/len(df)*100,5),'%')

Porcentaje de nulos por columna:
callsign 0.0 %
origin 30.46714 %
destination 25.67108 %
day 0.0 %
latitude_1 0.0 %
longitude_1 0.0 %
latitude_2 0.00238 %
longitude_2 0.00238 %


Se observa que el porcentaje de nulos para el origen y destinación de vuelos es bastante elevado, sin embargo no es así con el resto de variables donde los nulos son próximos al 0%.

A continuación, se preprocesarán los datos para poder obtener resultados más limpios y concisos, empezando por la columna ***callsign***. De este campo se puede obtener la aerolínea que opera el vuelo con los 3 primeros dígitos y, por tanto, también el país al que pertenece la aerolínea. Estos 3 primeros dígitos corresponden al código **ICAO**, los cuales se pueden codificar al nombre de la aerolínea utilizando la siguiente base de datos https://en.wikipedia.org/wiki/List_of_airline_codes. Se crea un excel con los códigos, se importa como un data frame y se realiza un ***join*** a la tabla de datos que se está usando.

***NOTA IMPORTANTE***: Este archivo contiene códigos de aerolíneas duplicados cuando este código es único para cada areolína. Esto es probablemente debido a dos motivos, aparición de nuevas areolíneas que toman códigos de areolíneas que ya han cerrado o directamente errores en la fuente de información. En la columna comentarios de la tabla, en ocasiones encontramos *'defunct'*, pero en otras no. No se puede dar con un método automático para limpiar los más de 200 códigos con más de 1 registro en el archivo, la única manera fiable es estudiar caso a caso. Se ha realizado un trabajo de limpieza manual sobre estos datos a medida que se avanzaba en la limpieza de los datos y análisis.

In [9]:
# creamos la columna auxiliar con el código de la aerolinea
df['ICAO'] = df['callsign'].apply(lambda x: x[0:3])

In [10]:
df.head()

,callsign,origin,destination,day,latitude_1,longitude_1,latitude_2,longitude_2,ICAO
0,HVN19,YMML,LFPG,2019-01-01 00:00:00+00:00,-37.659485,144.804421,48.995316,2.610802,HVN
1,CCA839,YMML,LEBL,2019-01-01 00:00:00+00:00,-37.692123,144.841997,41.036124,2.063557,CCA
2,CES219,YSSY,EDDF,2019-01-01 00:00:00+00:00,-33.930908,151.171987,50.045563,8.588923,CES
3,AEA040,LEMD,LEMD,2019-01-01 00:00:00+00:00,40.534756,-3.575426,40.475728,-3.538347,AEA
4,CXA825,YSSY,LFPG,2019-01-01 00:00:00+00:00,-33.954254,151.178041,48.996091,2.625805,CXA


In [11]:
df_icao_code = pd.read_excel('aux_files/IATA_airlines_code.xlsx',sheet_name='airline_code',)

In [12]:
df_icao_code.head()

,ICAO,Airline,Call sign,Country/Region,Comments
0,GNL,135 Airways,GENERAL,United States,NaN
1,TBS,Timbis Air,NaN,Kenya,NaN
2,WYT,2 Sqn No 1 Elementary Flying Training School,WYTON,United Kingdom,Royal Air Force
3,TFU,213th Flight Unit,THJY,Russia,State Airline
4,CHD,223rd Flight Unit,CHKALOVSK-AVIA,Russia,State Airline


In [13]:
df_icao_code.drop(['Call sign','Comments'],inplace=True,axis=1)
df_icao_code.columns = ['ICAO','Airline','Airline_country']

In [14]:
df_m = df.merge(df_icao_code, how='inner', on='ICAO')

In [15]:
df_m.drop('callsign',axis=1,inplace=True)
df_m

,origin,destination,day,latitude_1,longitude_1,latitude_2,longitude_2,ICAO,Airline,Airline_country
0,YMML,LFPG,2019-01-01 00:00:00+00:00,-37.659485,144.804421,48.995316,2.610802,HVN,Vietnam Airlines,Vietnam
1,YSSY,EGLL,2019-01-01 00:00:00+00:00,-33.955399,151.178321,51.464996,-0.433292,HVN,Vietnam Airlines,Vietnam
2,NaN,EDDF,2019-01-01 00:00:00+00:00,36.775011,126.464792,50.043050,8.578244,HVN,Vietnam Airlines,Vietnam
3,RCTP,NaN,2019-01-01 00:00:00+00:00,25.112961,121.267700,20.957585,119.816745,HVN,Vietnam Airlines,Vietnam
4,RJBB,NaN,2019-01-01 00:00:00+00:00,34.434540,135.254647,21.692505,119.759878,HVN,Vietnam Airlines,Vietnam
...,...,...,...,...,...,...,...,...,...,...
1816555,NaN,NaN,2019-01-31 00:00:00+00:00,45.259506,-73.901040,23.879974,-81.006622,TSS,Tri-State Aero,United States
1816556,LIRE,LIRE,2019-01-31 00:00:00+00:00,41.601425,12.460431,41.663648,12.427773,RIL,Regional Air,Mauritania
1816557,NaN,YAMB,2019-01-31 00:00:00+00:00,-31.552129,151.793427,-27.626798,152.686985,TBO,Aero Taxi de Los Cabos,Mexico
1816558,YMEN,NaN,2019-01-31 00:00:00+00:00,-37.735104,144.901727,-35.963512,143.439370,LEP,Air Costa,India


En este momento, destaca la existencia de códigos **ICAO** sin compañía aérea o código asociado. Investigando, se resuelve que esta tipología de vuelos corresponden a vuelos privados, por tanto, al no ser objeto del estudio en este trabajo se eliminan aquellas filas sin código **ICAO** asociado o país. Realizado este paso, se observa que el porcentaje de nulos solo sigue existiendo en ***origin*** y ***destination***.

In [16]:
print('Porcentaje de nulos por columna:')
for e in df_m.columns: 
    print(e,np.round(df_m[e].isnull().sum()/len(df_m)*100,5),'%')

Porcentaje de nulos por columna:
origin 31.94043 %
destination 28.36251 %
day 0.0 %
latitude_1 0.0 %
longitude_1 0.0 %
latitude_2 0.0016 %
longitude_2 0.0016 %
ICAO 0.0 %
Airline 0.0 %
Airline_country 0.00022 %


Antes de tratar estos nulos, se enriquecerá el data set usando la codificación de 4 letras que indica el aeropuerto, su ciudad correspondiente y país. Para ello, se utiliza el archivo *'airports.csv'* del enlace https://ourairports.com/data/ . Para este trabajo, la tabla ha sido modificada en excel en algunos puntos para simplificar su carga evitando errores innecesarios y, además, se han borrado campos que no aportan valor al estudio.

In [17]:
air = pd.read_csv('aux_files/airports.csv',encoding='utf-8',sep=';',keep_default_na=False)

In [18]:
air.head()

,ident,type,name,continent,iso_country,iso_region,municipality
0,00A,heliport,Total Rf Heliport,NA,US,US-PA,Bensalem
1,00AA,small_airport,Aero B Ranch Airport,NA,US,US-KS,Leoti
2,00AK,small_airport,Lowell Field,NA,US,US-AK,Anchor Point
3,00AL,small_airport,Epps Airpark,NA,US,US-AL,Harvest
4,00AR,closed,Newport Hospital & Clinic Heliport,NA,US,US-AR,Newport


In [19]:
air = air.drop(['type','name','iso_region'],axis=1)

In [20]:
df_m2 = pd.merge(left=df_m,right=air,how='left',left_on='origin',right_on='ident')

In [21]:
df_m2 = df_m2.drop(['ident'],axis=1)

In [22]:
df_m2.columns = ['origin', 'destination', 'day', 'latitude_1', 'longitude_1',
                 'latitude_2', 'longitude_2', 'ICAO', 'Airline', 'Airline_country',
                 'continent_o', 'iso_country_o', 'municipality_o']

In [23]:
df_m2.head()

,origin,destination,day,latitude_1,longitude_1,latitude_2,longitude_2,ICAO,Airline,Airline_country,continent_o,iso_country_o,municipality_o
0,YMML,LFPG,2019-01-01 00:00:00+00:00,-37.659485,144.804421,48.995316,2.610802,HVN,Vietnam Airlines,Vietnam,OC,AU,Melbourne
1,YSSY,EGLL,2019-01-01 00:00:00+00:00,-33.955399,151.178321,51.464996,-0.433292,HVN,Vietnam Airlines,Vietnam,OC,AU,Sydney
2,NaN,EDDF,2019-01-01 00:00:00+00:00,36.775011,126.464792,50.043050,8.578244,HVN,Vietnam Airlines,Vietnam,NaN,NaN,NaN
3,RCTP,NaN,2019-01-01 00:00:00+00:00,25.112961,121.267700,20.957585,119.816745,HVN,Vietnam Airlines,Vietnam,AS,TW,Taipei
4,RJBB,NaN,2019-01-01 00:00:00+00:00,34.434540,135.254647,21.692505,119.759878,HVN,Vietnam Airlines,Vietnam,AS,JP,Osaka


In [24]:
df_m2 = pd.merge(left=df_m2,right=air,how='left',left_on='destination',right_on='ident')

In [25]:
df_m2 = df_m2.drop(['ident'],axis=1)

In [26]:
df_m2.columns = ['origin', 'destination', 'day', 'latitude_1', 'longitude_1',
                 'latitude_2', 'longitude_2', 'ICAO', 'Airline', 'Airline_country',
                 'continent_o', 'iso_country_o', 'municipality_o', 
                 'continent_d','iso_country_d', 'municipality_d']

In [27]:
df_m2.head()

,origin,destination,day,latitude_1,longitude_1,latitude_2,longitude_2,ICAO,Airline,Airline_country,continent_o,iso_country_o,municipality_o,continent_d,iso_country_d,municipality_d
0,YMML,LFPG,2019-01-01 00:00:00+00:00,-37.659485,144.804421,48.995316,2.610802,HVN,Vietnam Airlines,Vietnam,OC,AU,Melbourne,EU,FR,Paris
1,YSSY,EGLL,2019-01-01 00:00:00+00:00,-33.955399,151.178321,51.464996,-0.433292,HVN,Vietnam Airlines,Vietnam,OC,AU,Sydney,EU,GB,London
2,NaN,EDDF,2019-01-01 00:00:00+00:00,36.775011,126.464792,50.043050,8.578244,HVN,Vietnam Airlines,Vietnam,NaN,NaN,NaN,EU,DE,Frankfurt am Main
3,RCTP,NaN,2019-01-01 00:00:00+00:00,25.112961,121.267700,20.957585,119.816745,HVN,Vietnam Airlines,Vietnam,AS,TW,Taipei,NaN,NaN,NaN
4,RJBB,NaN,2019-01-01 00:00:00+00:00,34.434540,135.254647,21.692505,119.759878,HVN,Vietnam Airlines,Vietnam,AS,JP,Osaka,NaN,NaN,NaN


Como indicado anteriormente, el campo *day* es de tipo string así que se modifica a tipo fecha con el formato ***'yyyy-mm-dd'***

In [28]:
df_m2['day'] =  df_m2['day'].apply(lambda x: x[0:11])
df_m2['day'] = pd.to_datetime(df_m2['day'])

In [29]:
df_m2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1816560 entries, 0 to 1816559
Data columns (total 16 columns):
 #   Column           Dtype         
---  ------           -----         
 0   origin           object        
 1   destination      object        
 2   day              datetime64[ns]
 3   latitude_1       float64       
 4   longitude_1      float64       
 5   latitude_2       float64       
 6   longitude_2      float64       
 7   ICAO             object        
 8   Airline          object        
 9   Airline_country  object        
 10  continent_o      object        
 11  iso_country_o    object        
 12  municipality_o   object        
 13  continent_d      object        
 14  iso_country_d    object        
 15  municipality_d   object        
dtypes: datetime64[ns](1), float64(4), object(11)
memory usage: 235.6+ MB


Ahora se procesan los nulos en las columnas ***origin*** y ***destination***. Para ello, primeramente se determina el país al que pertenecen las coordenadas de latitud y longitud asociadas. Se entiende que el primer lugar y último lugar de detección corresponden al origen y destino del vuelo. Este enriquecimiento permitirá determinar la localización de aquellos campos con nulos.

Para ello se utiliza la función **reverse_geocoder.search()**. A continuación, se muestra un ejemplo de su funcionamiento, donde a partir de la latitud y la longitud se determina el nombre de la ciudad y el país.

In [55]:
# ejemplo para un campo con NaN en nuestro data frame correspodiente al destino de la 4a fila
import reverse_geocoder as rg
coordinates = (48.995316,2.610802)
print('City: ', list(rg.search(coordinates)[0].values())[2])
print('Region: ', list(rg.search(coordinates)[0].values())[3])
print('Iso_country: ', list(rg.search(coordinates)[0].values())[5])
print('\n')
coordinates = (20.957585,119.816745)
print('City: ', list(rg.search(coordinates)[0].values())[2])
print('Region: ', list(rg.search(coordinates)[0].values())[3])
print('Iso_country: ', list(rg.search(coordinates)[0].values())[5])

Loading formatted geocoded file...
City:  Mitry-Mory
Region:  Ile-de-France
Iso_country:  FR


City:  Hengchun
Region:  Taiwan
Iso_country:  TW


Primeramente, se eliminan las filas con coordenadas nulas o con origen y/o destino nulo si y solo si también las latitudes o longitudes asociadas son nulas.

In [106]:
df_clean = df_m2.copy()

In [107]:
rows_o_del = df_m2[df_m2['origin'].isnull() & (df_m2['latitude_1'].isnull() | df_m2['longitude_1'].isnull())].index

In [108]:
rows_d_del = df_m2[df_m2['destination'].isnull() & (df_m2['latitude_2'].isnull() | df_m2['longitude_2'].isnull())].index

In [109]:
print(rows_o_del)
print(rows_d_del)

Int64Index([], dtype='int64')
Int64Index([189840, 190143, 490005, 579456, 741019, 909354, 1205532, 1502712,
            1665148],
           dtype='int64')


In [110]:
df_clean.drop(rows_o_del, inplace=True)
df_clean.drop(rows_d_del, inplace=True)

In [111]:
df_clean.reset_index(drop=True, inplace=True)

In [112]:
df_clean.info() #hemos reducido en 9 las entradas sin origen o destino definidos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1816551 entries, 0 to 1816550
Data columns (total 16 columns):
 #   Column           Dtype         
---  ------           -----         
 0   origin           object        
 1   destination      object        
 2   day              datetime64[ns]
 3   latitude_1       float64       
 4   longitude_1      float64       
 5   latitude_2       float64       
 6   longitude_2      float64       
 7   ICAO             object        
 8   Airline          object        
 9   Airline_country  object        
 10  continent_o      object        
 11  iso_country_o    object        
 12  municipality_o   object        
 13  continent_d      object        
 14  iso_country_d    object        
 15  municipality_d   object        
dtypes: datetime64[ns](1), float64(4), object(11)
memory usage: 221.7+ MB


Después de este proceso se observa que siguen apareciendo parámetros de localización nulos, es decir, existen datos con una destinación registrada en ***destination*** pero no sus parámetros de localización por coordenadas. Como sigue habiendo nulos en algunos parámetros de latitud y longitud, los imputamos como **(-90,0)** (valor del polo sur) para que la función no dé error ya que no puede calcular lugares con nulos en sus valores. Después, estos registros se eliminan para que no induzca a errores de análisis futuros. Si comprobamos la localización de estas cordenadas, comprobamos que se tratan de las islas *Islas Georgia del Sur y Sandwich del Sur*, unas islas remotas controladas por el Reino Unido a donde solamente se llega por barco, por tanto, despues será facil eliminar estos registros, basta con buscar aquellas localidades con el código 'GS'.

Aprovechando este ejemplo de abajo, se comprueba que la función puede dar como resultado **''**, es decir, **un string vacío**. Al final, se tratarán también estos casos para que computen como nulos y comprobar la cantidad de los mismos.

In [113]:
coordinates = (-90,0)
print('City: ', list(rg.search(coordinates)[0].values())[2])
print('Region: ', list(rg.search(coordinates)[0].values())[3])
print('Iso_country: ', list(rg.search(coordinates)[0].values())[5])

City:  Grytviken
Region:  
Iso_country:  GS


In [114]:
df_clean['latitude_1'].fillna(-90, inplace=True)
df_clean['latitude_2'].fillna(-90, inplace=True)
df_clean['longitude_1'].fillna(0, inplace=True)
df_clean['longitude_2'].fillna(0, inplace=True)

Ahora procedemos a rellenar la información de los orígenes o destinos no definidos.

In [116]:
coordenates_o = list(zip(df_clean['latitude_1'],df_clean['longitude_1']))
coordenates_d = list(zip(df_clean['latitude_2'],df_clean['longitude_2']))
coor_o_res = rg.search(coordenates_o)
coor_d_res = rg.search(coordenates_d)

In [117]:
o_city = []
o_region = []
o_iso = []

d_city = []
d_region = []
d_iso = []

for loc in range(len(coor_o_res)):
    o_city.append(list(coor_o_res[loc].values())[2])
    o_region.append(list(coor_o_res[loc].values())[3])
    o_iso.append(list(coor_o_res[loc].values())[5])
    
    d_city.append(list(coor_d_res[loc].values())[2])
    d_region.append(list(coor_d_res[loc].values())[3])
    d_iso.append(list(coor_d_res[loc].values())[5])

In [118]:
df_locations = pd.DataFrame({'o_city':o_city, 'o_region':o_region, 'o_iso':o_iso, 'd_city':d_city, 'd_region':d_region, 'd_iso':d_iso})

In [119]:
df_locations

,o_city,o_region,o_iso,d_city,d_region,d_iso
0,Keilor Lodge,Victoria,AU,Mitry-Mory,Ile-de-France,FR
1,Botany,New South Wales,AU,Feltham,England,GB
2,Suisan,Chungcheongnam-do,KR,Morfelden-Walldorf,Hesse,DE
3,Taoyuan City,Taiwan,TW,Hengchun,Taiwan,TW
4,Kaizuka,Osaka,JP,Hengchun,Taiwan,TW
...,...,...,...,...,...,...
1816546,Beauharnois,Quebec,CA,Varadero,Matanzas,CU
1816547,Torvaianica,Latium,IT,Torvaianica,Latium,IT
1816548,Gloucester,New South Wales,AU,One Mile,Queensland,AU
1816549,Essendon North,Victoria,AU,Swan Hill,Victoria,AU


Recordamos los registros con aquellas localizaciones que se complementaron para no tener nulos correspondientes a los registros con el país **'GS'**. Se introducen los nulos en estos registros.

In [120]:
# origen
df_locations['o_city'].mask((df_locations['o_iso']=='GS'), np.nan, inplace=True)
df_locations['o_region'].mask((df_locations['o_iso']=='GS'), np.nan, inplace=True)
df_locations['o_iso'].mask((df_locations['o_iso']=='GS'), np.nan, inplace=True)

#destinación
df_locations['d_city'].mask((df_locations['d_iso']=='GS'), np.nan, inplace=True)
df_locations['d_region'].mask((df_locations['d_iso']=='GS'), np.nan, inplace=True)
df_locations['d_iso'].mask((df_locations['d_iso']=='GS'), np.nan, inplace=True)

In [121]:
df_clean2 = pd.concat([df_clean,df_locations],axis=1)

In [123]:
pd.set_option('display.max_columns', None)
df_clean2.head()

,origin,destination,day,latitude_1,longitude_1,latitude_2,longitude_2,ICAO,Airline,Airline_country,continent_o,iso_country_o,municipality_o,continent_d,iso_country_d,municipality_d,o_city,o_region,o_iso,d_city,d_region,d_iso
0,YMML,LFPG,2019-01-01,-37.659485,144.804421,48.995316,2.610802,HVN,Vietnam Airlines,Vietnam,OC,AU,Melbourne,EU,FR,Paris,Keilor Lodge,Victoria,AU,Mitry-Mory,Ile-de-France,FR
1,YSSY,EGLL,2019-01-01,-33.955399,151.178321,51.464996,-0.433292,HVN,Vietnam Airlines,Vietnam,OC,AU,Sydney,EU,GB,London,Botany,New South Wales,AU,Feltham,England,GB
2,NaN,EDDF,2019-01-01,36.775011,126.464792,50.043050,8.578244,HVN,Vietnam Airlines,Vietnam,NaN,NaN,NaN,EU,DE,Frankfurt am Main,Suisan,Chungcheongnam-do,KR,Morfelden-Walldorf,Hesse,DE
3,RCTP,NaN,2019-01-01,25.112961,121.267700,20.957585,119.816745,HVN,Vietnam Airlines,Vietnam,AS,TW,Taipei,NaN,NaN,NaN,Taoyuan City,Taiwan,TW,Hengchun,Taiwan,TW
4,RJBB,NaN,2019-01-01,34.434540,135.254647,21.692505,119.759878,HVN,Vietnam Airlines,Vietnam,AS,JP,Osaka,NaN,NaN,NaN,Kaizuka,Osaka,JP,Hengchun,Taiwan,TW


En este punto, aparece un nuevo aspecto a tratar, en algunos vuelos el origen y el destino tienen como origen y destino la misma ciudad o aeropuerto por motivo desconocido (condiciones climáticas, emergencia sanitaria, vuelos puntuales, fallo mecánica, vuelos pequeños entre localidades cercanas, movimientos entre islas, maniobras militares etc.). También se procede a suprimir estos registros. 

In [124]:
rows_cities_del = df_clean2[df_clean2['o_city']==df_clean2['d_city']].index

In [125]:
df_clean2[df_clean2['o_city']==df_clean2['d_city']]

,origin,destination,day,latitude_1,longitude_1,latitude_2,longitude_2,ICAO,Airline,Airline_country,continent_o,iso_country_o,municipality_o,continent_d,iso_country_d,municipality_d,o_city,o_region,o_iso,d_city,d_region,d_iso
918,RKPK,RKPK,2019-01-17,35.187012,128.936107,35.153643,128.940353,HVN,Vietnam Airlines,Vietnam,AS,KR,Busan,AS,KR,Busan,Kimhae,Gyeongsangnam-do,KR,Kimhae,Gyeongsangnam-do,KR
1474,RKPK,RKPK,2019-01-25,35.192840,128.937721,35.157321,128.939781,HVN,Vietnam Airlines,Vietnam,AS,KR,Busan,AS,KR,Busan,Kimhae,Gyeongsangnam-do,KR,Kimhae,Gyeongsangnam-do,KR
2186,SCTL,SCTL,2019-01-01,-35.340134,-71.478481,-35.344600,-71.557052,CCA,Air China,China,SA,CL,Talca,SA,CL,Talca,Talca,Maule,CL,Talca,Maule,CL
2187,SCTL,SCTL,2019-01-01,-35.343995,-71.503289,-35.359916,-71.564006,CCA,Air China,China,SA,CL,Talca,SA,CL,Talca,Talca,Maule,CL,Talca,Maule,CL
2442,SCTL,SCTL,2019-01-02,-35.344880,-71.551676,-35.311981,-71.533642,CCA,Air China,China,SA,CL,Talca,SA,CL,Talca,Talca,Maule,CL,Talca,Maule,CL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1816494,YSBK,YSBK,2019-01-28,-33.919310,150.995178,-33.919310,150.995121,OPA,Opal Air,Australia,OC,AU,Sydney,OC,AU,Sydney,Milperra,New South Wales,AU,Milperra,New South Wales,AU
1816499,NaN,KLSV,2019-01-29,36.339798,-114.904404,36.234865,-115.038426,SLV,Stella Aviation,Mauritania,NaN,NaN,NaN,NA,US,Las Vegas,Nellis Air Force Base,Nevada,US,Nellis Air Force Base,Nevada,US
1816521,EHLE,EHLE,2019-01-30,52.451752,5.511932,52.452667,5.515747,PHO,L&L Flygbildteknik,Sweden,EU,NL,Lelystad,EU,NL,Lelystad,Lelystad,Flevoland,NL,Lelystad,Flevoland,NL
1816544,NaN,KLSV,2019-01-31,36.334305,-114.912014,36.247101,-115.023880,UGL,Inter-Island Air,United States,NaN,NaN,NaN,NA,US,Las Vegas,Nellis Air Force Base,Nevada,US,Nellis Air Force Base,Nevada,US


In [126]:
df_clean2.drop(rows_cities_del, inplace=True)

In [127]:
df_clean2.reset_index(drop=True,inplace=True)

In [128]:
df_clean2.head()

,origin,destination,day,latitude_1,longitude_1,latitude_2,longitude_2,ICAO,Airline,Airline_country,continent_o,iso_country_o,municipality_o,continent_d,iso_country_d,municipality_d,o_city,o_region,o_iso,d_city,d_region,d_iso
0,YMML,LFPG,2019-01-01,-37.659485,144.804421,48.995316,2.610802,HVN,Vietnam Airlines,Vietnam,OC,AU,Melbourne,EU,FR,Paris,Keilor Lodge,Victoria,AU,Mitry-Mory,Ile-de-France,FR
1,YSSY,EGLL,2019-01-01,-33.955399,151.178321,51.464996,-0.433292,HVN,Vietnam Airlines,Vietnam,OC,AU,Sydney,EU,GB,London,Botany,New South Wales,AU,Feltham,England,GB
2,NaN,EDDF,2019-01-01,36.775011,126.464792,50.043050,8.578244,HVN,Vietnam Airlines,Vietnam,NaN,NaN,NaN,EU,DE,Frankfurt am Main,Suisan,Chungcheongnam-do,KR,Morfelden-Walldorf,Hesse,DE
3,RCTP,NaN,2019-01-01,25.112961,121.267700,20.957585,119.816745,HVN,Vietnam Airlines,Vietnam,AS,TW,Taipei,NaN,NaN,NaN,Taoyuan City,Taiwan,TW,Hengchun,Taiwan,TW
4,RJBB,NaN,2019-01-01,34.434540,135.254647,21.692505,119.759878,HVN,Vietnam Airlines,Vietnam,AS,JP,Osaka,NaN,NaN,NaN,Kaizuka,Osaka,JP,Hengchun,Taiwan,TW


En los resultados que se obtienen, se observa que la ***municipality*** obtenida de la tabla inicial puede o no coincidir con la ***city*** o ***region*** obtenidas dado que la librería *reverse_geocoder* nos da como resultado las localidades correspondientes a la zona asociada a las coordenadas. Así pues, para el Aeropuerto del Prat en Barcelona, podemos obtener como ciudad Viladecans o Castelldefels, cosa que es correcta, pero como es evidente estos vuelos en realidad corresponden al aeropuerto de Barcelona. Este hecho será tratado más adelante. 

Asimismo, aparecen casuísticas donde localidades en las que el país obtenido mediante coordenadas no coincide con el país del aeropuerto. Esto es debido a que la localización se encuentra justo entre la frontera entre dos países, como puede ser el aeropuerto de Ginebra en Suiza (en la frontera con Francia) o el aeropuerto de San Diego en Estados Unidos (en la frontera con México). Se analiza a continuación en que porcentaje de casos esto ocurre.

In [129]:
"""
Condición de aquellos registros sin nulos en el aeropuerto de origen y destino y cuyo país calculado con las coordenadas
no coincide con el país correcto
"""
no_match_o = df_clean2[df_clean2['iso_country_o'].notnull() & (df_clean2['iso_country_o']!=df_clean2['o_iso'])].index
no_match_d = df_clean2[df_clean2['iso_country_d'].notnull() & (df_clean2['iso_country_d']!=df_clean2['d_iso'])].index

# porcentaje de registros no repetidos sobre el total
no_iso_match = len(set(list(no_match_o)).union(set(list(no_match_d))))/len(df_clean2)*100
print('Percent of registers not matching countries: ',np.round(no_iso_match,3),'%')

Percent of registers not matching countries:  0.421 %


Se comprueba que se trata de una cantidad muy pequeña de registros sobre el total, por ello no se irá más lejos en este apartado del análisis. Asimismo, para disminuir aún más el % de error, se seleccionan para el dataframe final el código del país que se ha obtenido del aeropuerto y no el calculado para evitar algunos de estos errores. 

In [130]:
df_clean2

,origin,destination,day,latitude_1,longitude_1,latitude_2,longitude_2,ICAO,Airline,Airline_country,continent_o,iso_country_o,municipality_o,continent_d,iso_country_d,municipality_d,o_city,o_region,o_iso,d_city,d_region,d_iso
0,YMML,LFPG,2019-01-01,-37.659485,144.804421,48.995316,2.610802,HVN,Vietnam Airlines,Vietnam,OC,AU,Melbourne,EU,FR,Paris,Keilor Lodge,Victoria,AU,Mitry-Mory,Ile-de-France,FR
1,YSSY,EGLL,2019-01-01,-33.955399,151.178321,51.464996,-0.433292,HVN,Vietnam Airlines,Vietnam,OC,AU,Sydney,EU,GB,London,Botany,New South Wales,AU,Feltham,England,GB
2,NaN,EDDF,2019-01-01,36.775011,126.464792,50.043050,8.578244,HVN,Vietnam Airlines,Vietnam,NaN,NaN,NaN,EU,DE,Frankfurt am Main,Suisan,Chungcheongnam-do,KR,Morfelden-Walldorf,Hesse,DE
3,RCTP,NaN,2019-01-01,25.112961,121.267700,20.957585,119.816745,HVN,Vietnam Airlines,Vietnam,AS,TW,Taipei,NaN,NaN,NaN,Taoyuan City,Taiwan,TW,Hengchun,Taiwan,TW
4,RJBB,NaN,2019-01-01,34.434540,135.254647,21.692505,119.759878,HVN,Vietnam Airlines,Vietnam,AS,JP,Osaka,NaN,NaN,NaN,Kaizuka,Osaka,JP,Hengchun,Taiwan,TW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1797115,NaN,YSCN,2019-01-31,-34.014862,150.117636,-34.023867,150.713768,TCE,Trans-Colorado Airlines,United States,NaN,NaN,NaN,OC,AU,,Katoomba,New South Wales,AU,Narellan,New South Wales,AU
1797116,NaN,NaN,2019-01-31,45.259506,-73.901040,23.879974,-81.006622,TSS,Tri-State Aero,United States,NaN,NaN,NaN,NaN,NaN,NaN,Beauharnois,Quebec,CA,Varadero,Matanzas,CU
1797117,NaN,YAMB,2019-01-31,-31.552129,151.793427,-27.626798,152.686985,TBO,Aero Taxi de Los Cabos,Mexico,NaN,NaN,NaN,OC,AU,,Gloucester,New South Wales,AU,One Mile,Queensland,AU
1797118,YMEN,NaN,2019-01-31,-37.735104,144.901727,-35.963512,143.439370,LEP,Air Costa,India,OC,AU,Melbourne,NaN,NaN,NaN,Essendon North,Victoria,AU,Swan Hill,Victoria,AU


In [131]:
df_final = df_clean2.copy()

In [133]:
df_final[df_final['origin'].notnull() & df_final['iso_country_o'].isna()].head()

,origin,destination,day,latitude_1,longitude_1,latitude_2,longitude_2,ICAO,Airline,Airline_country,continent_o,iso_country_o,municipality_o,continent_d,iso_country_d,municipality_d,o_city,o_region,o_iso,d_city,d_region,d_iso
10228,RJNS,NaN,2019-01-01,34.760882,138.207378,33.095673,128.728784,CES,China Eastern Airlines,China,NaN,NaN,NaN,NaN,NaN,NaN,Shimada,Shizuoka,JP,Fukuecho,Nagasaki,JP
10482,RJNS,NaN,2019-01-02,34.758694,138.208294,32.727768,127.999347,CES,China Eastern Airlines,China,NaN,NaN,NaN,NaN,NaN,NaN,Shimada,Shizuoka,JP,Fukuecho,Nagasaki,JP
10982,RJNS,NaN,2019-01-04,34.768470,138.219452,32.881302,128.715271,CES,China Eastern Airlines,China,NaN,NaN,NaN,NaN,NaN,NaN,Shimada,Shizuoka,JP,Fukuecho,Nagasaki,JP
11038,RJNS,NaN,2019-01-04,34.764633,138.222750,33.159760,128.856995,CES,China Eastern Airlines,China,NaN,NaN,NaN,NaN,NaN,NaN,Shimada,Shizuoka,JP,Fukuecho,Nagasaki,JP
11285,RJNS,NaN,2019-01-05,34.736628,138.199310,32.852426,128.144027,CES,China Eastern Airlines,China,NaN,NaN,NaN,NaN,NaN,NaN,Sagara,Shizuoka,JP,Fukuecho,Nagasaki,JP


In [134]:
def get_origin_city(df):
    if pd.isna(df['origin']):
        return df['o_city']
    elif pd.isna(df['municipality_o']):
        return df['o_city']
    else:
        return df['municipality_o']
    
def get_destination_city(df):
    if pd.isna(df['destination']):
        return df['d_city']
    elif pd.isna(df['municipality_d']):
        return df['d_city']
    else:
        return df['municipality_d']
    
def get_origin_country(df):
    if pd.isna(df['origin']):
        return df['o_iso']
    elif pd.isna(df['iso_country_o']):
        return df['o_iso']
    else:
        return df['iso_country_o']
    
def get_destination_country(df):
    if pd.isna(df['destination']):
        return df['d_iso']
    elif pd.isna(df['iso_country_d']):
        return df['d_iso']
    else:
        return df['iso_country_d']

In [135]:
df_final['origin_city'] = df_final.apply(get_origin_city, axis=1)
df_final['destination_city'] = df_final.apply(get_destination_city, axis=1)
df_final['origin_country'] = df_final.apply(get_origin_country, axis=1)
df_final['destination_country'] = df_final.apply(get_destination_country, axis=1)

In [136]:
df_final.head()

,origin,destination,day,latitude_1,longitude_1,latitude_2,longitude_2,ICAO,Airline,Airline_country,continent_o,iso_country_o,municipality_o,continent_d,iso_country_d,municipality_d,o_city,o_region,o_iso,d_city,d_region,d_iso,origin_city,destination_city,origin_country,destination_country
0,YMML,LFPG,2019-01-01,-37.659485,144.804421,48.995316,2.610802,HVN,Vietnam Airlines,Vietnam,OC,AU,Melbourne,EU,FR,Paris,Keilor Lodge,Victoria,AU,Mitry-Mory,Ile-de-France,FR,Melbourne,Paris,AU,FR
1,YSSY,EGLL,2019-01-01,-33.955399,151.178321,51.464996,-0.433292,HVN,Vietnam Airlines,Vietnam,OC,AU,Sydney,EU,GB,London,Botany,New South Wales,AU,Feltham,England,GB,Sydney,London,AU,GB
2,NaN,EDDF,2019-01-01,36.775011,126.464792,50.043050,8.578244,HVN,Vietnam Airlines,Vietnam,NaN,NaN,NaN,EU,DE,Frankfurt am Main,Suisan,Chungcheongnam-do,KR,Morfelden-Walldorf,Hesse,DE,Suisan,Frankfurt am Main,KR,DE
3,RCTP,NaN,2019-01-01,25.112961,121.267700,20.957585,119.816745,HVN,Vietnam Airlines,Vietnam,AS,TW,Taipei,NaN,NaN,NaN,Taoyuan City,Taiwan,TW,Hengchun,Taiwan,TW,Taipei,Hengchun,TW,TW
4,RJBB,NaN,2019-01-01,34.434540,135.254647,21.692505,119.759878,HVN,Vietnam Airlines,Vietnam,AS,JP,Osaka,NaN,NaN,NaN,Kaizuka,Osaka,JP,Hengchun,Taiwan,TW,Osaka,Hengchun,JP,TW


Se comprueba de nuevo los nulos por columna para saber si se ha ejecutado correctamente. Se identifica que efectivamente no hay nulos en ninguno de los campos que nos interesan. Cabe destacar que aún están pendientes de procesamiento los strings vacíos de la función de localización automática, sin embargo, estos solo se encuentran en el campo ***region*** o ***city***.

In [137]:
print('Porcentaje de nulos por columna:')
for e in df_final.columns: 
    print(e,np.round(df_final[e].isnull().sum()/len(df_final)*100,5),'%')

Porcentaje de nulos por columna:
origin 32.19117 %
destination 28.62602 %
day 0.0 %
latitude_1 0.0 %
longitude_1 0.0 %
latitude_2 0.0 %
longitude_2 0.0 %
ICAO 0.0 %
Airline 0.0 %
Airline_country 0.00022 %
continent_o 32.36862 %
iso_country_o 32.36862 %
municipality_o 32.36862 %
continent_d 28.8062 %
iso_country_d 28.8062 %
municipality_d 28.8062 %
o_city 0.0 %
o_region 0.0 %
o_iso 0.0 %
d_city 0.00111 %
d_region 0.00111 %
d_iso 0.00111 %
origin_city 0.0 %
destination_city 0.0 %
origin_country 0.0 %
destination_country 0.0 %


Eliminamos las columnas redundantes: ***origin, destination, ICAO, continent_o, iso_country_o, municipality_o, continent_d, iso_country_d, municipality_d, o_city, o_iso, d_city, d_iso***

In [138]:
df_final_2 = df_final.copy()

In [139]:
df_final_2.drop(['origin', 'destination','ICAO','continent_o', 'iso_country_o', 'municipality_o', 'continent_d', 
               'iso_country_d', 'municipality_d', 'o_city', 'o_iso','d_city','d_iso'],axis=1,inplace=True)

In [140]:
df_final_2.columns = ['day', 'latitude_1', 'longitude_1', 'latitude_2', 'longitude_2',
                    'Airline', 'Airline_country', 'origin_region', 'destination_region', 
                    'origin_city','destination_city', 'origin_country', 'destination_country']

In [141]:
df_final_2.head()

,day,latitude_1,longitude_1,latitude_2,longitude_2,Airline,Airline_country,origin_region,destination_region,origin_city,destination_city,origin_country,destination_country
0,2019-01-01,-37.659485,144.804421,48.995316,2.610802,Vietnam Airlines,Vietnam,Victoria,Ile-de-France,Melbourne,Paris,AU,FR
1,2019-01-01,-33.955399,151.178321,51.464996,-0.433292,Vietnam Airlines,Vietnam,New South Wales,England,Sydney,London,AU,GB
2,2019-01-01,36.775011,126.464792,50.043050,8.578244,Vietnam Airlines,Vietnam,Chungcheongnam-do,Hesse,Suisan,Frankfurt am Main,KR,DE
3,2019-01-01,25.112961,121.267700,20.957585,119.816745,Vietnam Airlines,Vietnam,Taiwan,Taiwan,Taipei,Hengchun,TW,TW
4,2019-01-01,34.434540,135.254647,21.692505,119.759878,Vietnam Airlines,Vietnam,Osaka,Taiwan,Osaka,Hengchun,JP,TW


A continuación, se tratarán los códigos de los países por su nombre y su continente. Para ello, se utiliza un archivo creado a partir de esta fuente https://en.wikipedia.org/wiki/List_of_sovereign_states_and_dependent_territories_by_continent_(data_file), donde se relaciona el código de dos dígitos de un país con su nombre y el continente. Algunos países pueden pertenecer a dos continentes a la misma vez, por ejemplo, Turquía figura como país europeo y asiático. Para evitar duplicidades de registros, se decanta por uno de los dos continentes en todos los casos.

In [142]:
df_country_cont = pd.read_csv('aux_files/country-and-continent-codes-list-csv.csv',sep=';',keep_default_na=False)

In [143]:
df_country_cont

,Continent_Name,Country_Name,Two_Letter_Country_Code
0,Asia,Afghanistan,AF
1,Europe,Aland Islands,AX
2,Europe,Albania,AL
3,Africa,Algeria,DZ
4,Oceania,American Samoa,AS
...,...,...,...
250,Africa,Western Sahara,EH
251,Asia,Yemen,YE
252,Africa,Zambia,ZM
253,Africa,Zimbabwe,ZW


In [144]:
df_final_2 = pd.merge(left=df_final_2, right=df_country_cont, how='left', 
                      left_on='origin_country',right_on='Two_Letter_Country_Code')

In [145]:
df_final_2.drop(['Two_Letter_Country_Code','origin_country'], axis=1, inplace=True)

In [146]:
df_final_2.columns = ['day', 'latitude_1', 'longitude_1', 'latitude_2', 'longitude_2',
                      'Airline', 'Airline_country', 'origin_region', 'destination_region',
                      'origin_city', 'destination_city', 'destination_country',
                      'origin_continent', 'origin_country']

In [147]:
df_final_2 = pd.merge(left=df_final_2, right=df_country_cont, how='left', 
                      left_on='destination_country',right_on='Two_Letter_Country_Code')

In [148]:
df_final_2.drop(['Two_Letter_Country_Code','destination_country'], axis=1, inplace=True)

In [149]:
df_final_2.columns = ['day', 'latitude_1', 'longitude_1', 'latitude_2', 'longitude_2',
                      'Airline', 'Airline_country', 'origin_region', 'destination_region',
                      'origin_city', 'destination_city', 'origin_continent', 'origin_country',
                      'destination_continent', 'destination_country']

In [150]:
df_final_2.head()

,day,latitude_1,longitude_1,latitude_2,longitude_2,Airline,Airline_country,origin_region,destination_region,origin_city,destination_city,origin_continent,origin_country,destination_continent,destination_country
0,2019-01-01,-37.659485,144.804421,48.995316,2.610802,Vietnam Airlines,Vietnam,Victoria,Ile-de-France,Melbourne,Paris,Oceania,Australia,Europe,France
1,2019-01-01,-33.955399,151.178321,51.464996,-0.433292,Vietnam Airlines,Vietnam,New South Wales,England,Sydney,London,Oceania,Australia,Europe,United Kingdom of Great Britain & Northern Ire...
2,2019-01-01,36.775011,126.464792,50.043050,8.578244,Vietnam Airlines,Vietnam,Chungcheongnam-do,Hesse,Suisan,Frankfurt am Main,Asia,South Korea,Europe,Germany
3,2019-01-01,25.112961,121.267700,20.957585,119.816745,Vietnam Airlines,Vietnam,Taiwan,Taiwan,Taipei,Hengchun,Asia,Taiwan,Asia,Taiwan
4,2019-01-01,34.434540,135.254647,21.692505,119.759878,Vietnam Airlines,Vietnam,Osaka,Taiwan,Osaka,Hengchun,Asia,Japan,Asia,Taiwan


En este punto, cerca de finalizar el tratamiento, se debe volver a asignar a nulos los valores para las coordenadas que asignamos con (-90,0).

In [151]:
df_final_3 = df_final_2.copy()

In [152]:
df_final_3['latitude_1'].mask((df_final_3['latitude_1']==-90) & (df_final_3['longitude_1']==0), np.nan, inplace=True)
df_final_3['longitude_1'].mask(df_final_3['latitude_1'].isna(), np.nan, inplace=True)
df_final_3['latitude_2'].mask((df_final_3['latitude_2']==-90) & (df_final_3['longitude_2']==0), np.nan, inplace=True)
df_final_3['longitude_2'].mask(df_final_3['latitude_2'].isna(), np.nan, inplace=True)

También se sustituyen los strings vacíos por nulos los cuales falsean ese resultado.

In [154]:
df_final_3.replace({'': np.nan}, inplace=True)

Se comprueban finalmente los nulos en nuestro data frame procesado. Ya se puede observar que no hay nulos en nuestro data frame y por tanto se puede dar por finalizada la fase de comprensión, análisis y limpieza de datos. 

In [155]:
print('Porcentaje de nulos por columna:')
for e in df_final_3.columns: 
    print(e,np.round(df_final_3[e].isnull().sum()/len(df_final_3)*100,5),'%')

Porcentaje de nulos por columna:
day 0.0 %
latitude_1 0.0 %
longitude_1 0.0 %
latitude_2 0.00111 %
longitude_2 0.00111 %
Airline 0.0 %
Airline_country 0.00022 %
origin_region 0.95336 %
destination_region 2.0027 %
origin_city 0.21284 %
destination_city 0.47693 %
origin_continent 0.0 %
origin_country 0.0 %
destination_continent 0.0 %
destination_country 0.0 %


Se comprueba que práctiamente se ha conseguido un **porcentaje de 0% de nulos** en casi todas las columnas de nuestro data frame final que nos van a ser útiles en posteriores análisis. En este punto, se considera que el data frame está apropiadamente limpio y listo para extraer la información que queramos sobre los vuelos en el mundo desde Enero de 2019 a Abril de 2021. Según el tipo de analisis que se quiera realizar, habrá que afinar en consecuencia los registros nulos (unificar ciudades y regiones).

Para obtener el resto de archivos (meses) limpios, será necesario iterar para cada unos de ellos siguiendo estos mismos pasos. No podemos computar todos los dataframes a la vez debido a las limitaciones de potencia en computación que nos restrigen. Para ello, hemos preparado otro archivo, **"2 - Procesamiento global de los datos para el análisis"**, con los pasos de este *notebook* pero comprimida en un bucle y sin los pasos intermedios mostrados para explicar el razonamiento propio llevado a cabo.

Como último apunte, para este data frame, se comprueba la cantidad de registros que tenemos con los que finalmente tenemos en nuestro data frame. Si se realiza el cálculo, se resuelve que disponemos del **83,76%** de los registros iniciales.

In [156]:
print('Porcentaje de registros que hemos mantenido respecto al data frame inicial: ')
print(np.round((len(df_final_3)/len(df))*100,2),'%')

Porcentaje de registros que hemos mantenido respecto al data frame inicial: 
83.76 %


Si se centrara, por ejemplo, en el análisis de vuelos con origen o destino en España, sería posible obtener cuantos vuelos de los totales representan en este data set de Enero de 2019 los vuelos que han pasado por este territorio. Si se realiza el cálculo se obtiene que suponen el 5% de vuelos en el mundo en ese mes.

In [157]:
esp_fl = df_final_3[(df_final_3['origin_country']=='Spain') | (df_final_3['destination_country']=='Spain')]
print('Total vuelos en España en Enero de 2019: ', len(esp_fl))
print('Porcentaje de registros pertenecientes a vuelos en España: ',np.round((len(esp_fl)/len(df_final_3))*100,2),'%')

Total vuelos en España en Enero de 2019:  90594
Porcetaje de registros pertenecientes a vuelos en España:  5.04 %


Finalmente, se transforma el dataframe procesado en un nuevo .csv para su uso en la segunda etapa del proyecto de análisis de los datos.

In [158]:
df_final_3.to_csv('input_data/1_clean.csv.gz', sep=',', index=False,encoding='utf-8')

En el siguiente apartado, se analizarán las variables finales que se han obtenido de este data frame final y, en base a los datos, se responderá a las preguntas formuladas al inicio del proyecto.